In [62]:
import contextlib
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup


df = pd.read_csv('../data/raw/cancillerias.csv')

df['Nombre'] = df['Nombre'].str.upper()

nombres = df['Nombre'].to_list()

data_list = []

for i in range(len(nombres)):

    data = {
        'query': f'{nombres[i]}',
        'find': 'Search',
    }

    response = requests.post('https://www.funcionpublica.gov.co/dafpIndexerBHV/hvSigep/index', data=data)

    html = response.text

    # Parse html
    soup = BeautifulSoup(html, 'html.parser')

    with contextlib.suppress(Exception):

        table_url = soup.table.td.a

        url = table_url.get('href')

        table_td = soup.table.td

        data_raw = [span.text for span in table_td.find_all('span')]

        data_raw.pop(-1)

        data_clean = [x.replace('\n', '').replace('\t', '').replace('\r', '') for x in data_raw]

        data_clean.append(url)

        data_list.append(data_clean)

# Convert list to dataframe and assign column names
df_list = pd.DataFrame(data_list, columns = ['Nombre', 'Cargo_General', 'Dependencia_General', 'Url'])

df_list['Nombre'] = df_list['Nombre'].str.upper()

df_list

,Nombre,Cargo_General,Dependencia_General,Url
0,ANNA MARIA GADZIEMSKA,Servidor Público,MINISTERIO DE RELACIONES EXTERIORES,https://www.funcionpublica.gov.co/web/sigep/hd...
1,ASSAD JOSE JATER PEÑA,Servidor Público,MINISTERIO DE RELACIONES EXTERIORES,https://www.funcionpublica.gov.co/web/sigep/hd...
2,BETTY ISABEL JAIMES SUAREZ,Servidor Público,MINISTERIO DE RELACIONES EXTERIORES,https://www.funcionpublica.gov.co/web/sigep/hd...
3,CRISTIAN JHOVANY QUITIAN CASTRILLON,Servidor Público,MINISTERIO DE RELACIONES EXTERIORES,https://www.funcionpublica.gov.co/web/sigep/hd...
4,KAREN GIRON ABADIA,Servidor Público,MINISTERIO DE RELACIONES EXTERIORES,https://www.funcionpublica.gov.co/web/sigep/hd...
...,...,...,...,...
482,MARIA TERESA MELLIZO ORTIZ,Servidor Público,MINISTERIO DE RELACIONES EXTERIORES,https://www.funcionpublica.gov.co/web/sigep/hd...
483,MARJORIE ANDREA GONZALEZ RAMIREZ,Servidor Público,MINISTERIO DE RELACIONES EXTERIORES,https://www.funcionpublica.gov.co/web/sigep/hd...
484,MONICA BELTRAN ESPITIA,Servidor Público,MINISTERIO DE RELACIONES EXTERIORES,https://www.funcionpublica.gov.co/web/sigep/hd...
485,RODRIGO ALBERTO VILLEGAS RIVILLAS,Servidor Público,MINISTERIO DE RELACIONES EXTERIORES,https://www.funcionpublica.gov.co/web/sigep/hd...


In [63]:
df = pd.read_csv('../data/raw/cancillerias.csv')

df_join = df.merge(df_list, how="left", on="Nombre")

df_join.to_csv('../data/processed/cancilleria_extendida.csv', index=False)

df_join

,Nombre,Cargo,Dependencia,Continente,País,Ciudad,Correo,Número,CP,Grupo_Trabajo,Cargo_General,Dependencia_General,Url
0,ANNA MARIA GADZIEMSKA,AUXILIAR ADMINISTRATIVO (LOCAL,EMBAJADA DE COLOMBIA ANTE EL GOBIERNO DE LA RE...,EUROPA,POLONIA,VARSOVIA,anna.gadziemska@cancilleria.gov.co,(+57-601) 3814000,4301,EMBAJADA DE COLOMBIA ANTE EL GOBIERNO DE LA RE...,Servidor Público,MINISTERIO DE RELACIONES EXTERIORES,https://www.funcionpublica.gov.co/web/sigep/hd...
1,ASSAD JOSE JATER PEÑA,EMBAJADOR,EMBAJADA DE COLOMBIA ANTE EL GOBIERNO DE LA RE...,EUROPA,POLONIA,VARSOVIA,assad.jater@cancilleria.gov.co,(+57-601) 3814000,NaN,EMBAJADA DE COLOMBIA ANTE EL GOBIERNO DE LA RE...,Servidor Público,MINISTERIO DE RELACIONES EXTERIORES,https://www.funcionpublica.gov.co/web/sigep/hd...
2,BETTY ISABEL JAIMES SUAREZ,AUXILIAR DE MISION DIPLOMATICA,EMBAJADA DE COLOMBIA ANTE EL GOBIERNO DE LA RE...,EUROPA,POLONIA,VARSOVIA,betty.jaimes@cancilleria.gov.co,(+57-601) 3814000,NaN,EMBAJADA DE COLOMBIA ANTE EL GOBIERNO DE LA RE...,Servidor Público,MINISTERIO DE RELACIONES EXTERIORES,https://www.funcionpublica.gov.co/web/sigep/hd...
3,CRISTIAN JHOVANY QUITIAN CASTRILLON,TERCER SECRETARIO,EMBAJADA DE COLOMBIA ANTE EL GOBIERNO DE LA RE...,EUROPA,POLONIA,VARSOVIA,cristian.quitian@cancilleria.gov.co,(+57-601) 3814000,NaN,EMBAJADA DE COLOMBIA ANTE EL GOBIERNO DE LA RE...,Servidor Público,MINISTERIO DE RELACIONES EXTERIORES,https://www.funcionpublica.gov.co/web/sigep/hd...
4,KAREN GIRON ABADIA,SEGUNDO SECRETARIO,EMBAJADA DE COLOMBIA ANTE EL GOBIERNO DE LA RE...,EUROPA,POLONIA,VARSOVIA,karen.giron@cancilleria.gov.co,(+57-601) 3814000,4819,EMBAJADA DE COLOMBIA ANTE EL GOBIERNO DE LA RE...,Servidor Público,MINISTERIO DE RELACIONES EXTERIORES,https://www.funcionpublica.gov.co/web/sigep/hd...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,MARIA TERESA MELLIZO ORTIZ,AUXILIAR DE MISION DIPLOMATICA,CONSULADO EN MONTREAL (CANADA),AMERICA DEL NORTE,CANADA,MONTREAL,maria.mellizo@cancilleria.gov.co,(+57-601) 3814000,NaN,CONSULADO EN MONTREAL (CANADA),Servidor Público,MINISTERIO DE RELACIONES EXTERIORES,https://www.funcionpublica.gov.co/web/sigep/hd...
513,MARJORIE ANDREA GONZALEZ RAMIREZ,SEGUNDO SECRETARIO,CONSULADO EN CALGARY (CANADA),AMERICA DEL NORTE,CANADA,CALGARY,marjorie.gonzalez@cancilleria.gov.co,(+57-601) 3814000,NaN,CONSULADO EN CALGARY (CANADA),Servidor Público,MINISTERIO DE RELACIONES EXTERIORES,https://www.funcionpublica.gov.co/web/sigep/hd...
514,MONICA BELTRAN ESPITIA,ENCARGADO DE NEGOCIOS A.I.,EMBAJADA DE COLOMBIA ANTE EL GOBIERNO DE CANADA,AMERICA DEL NORTE,CANADA,OTTAWA,monica.beltran@cancilleria.gov.co,(+57-601) 3814000,4293,EMBAJADA DE COLOMBIA ANTE EL GOBIERNO DE CANADA,Servidor Público,MINISTERIO DE RELACIONES EXTERIORES,https://www.funcionpublica.gov.co/web/sigep/hd...
515,RODRIGO ALBERTO VILLEGAS RIVILLAS,CONSEJERO DE RELACIONES EXTERI,EMBAJADA DE COLOMBIA ANTE EL GOBIERNO DE CANADA,AMERICA DEL NORTE,CANADA,OTTAWA,rodrigo.villegas@cancilleria.gov.co,(+57-601) 3814000,4294,EMBAJADA DE COLOMBIA ANTE EL GOBIERNO DE CANADA,Servidor Público,MINISTERIO DE RELACIONES EXTERIORES,https://www.funcionpublica.gov.co/web/sigep/hd...
